> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.11 - Data Model

## Exercícios

#### Q1. `dataclass`
Exercício adaptado de [codechalleng.es/bites/154/](https://codechalleng.es/bites/154/) e [codechalleng.es/bites/320/](https://codechalleng.es/bites/320/).

Neste desafio, você deve escrever uma `dataclass` chamada `Bite` que gerencia 3 atributos: `number`, `title` e `level`. Seus tipos são:
* `number` - `int`, 
* `title` - `str`, 
* `level` -  classe `Enum` chamada `BiteLevel` com os atributos `Beginner`, `Intermediate`, `Advanced`. 

Exemplo de dado: `{'number': 154, 'title': 'Escreva uma dataclass', 'level': BiteLevel.Intermediate}`

As características dessa classe são:
* O atributo`level` tem um valor padrão `BiteLevel.Beginner`
* Uma coleção de objetos `Bite` tem que ser ordenável somente pelo atributo `number`
* Implemente o método especial `__str__` para imprimir o Bite na forma `f'{number} - {title} ({level})'`

Teste sua classe executando o seguinte código:
```python
bites = []
bites.append(Bite(154, 'Escreva uma dataclass', 'Intermediate'))
bites.append(Bite(1, 'Some n valores'))
bites.append(Bite(37, 'Reescreva um loop com recursão', 'Intermediate'))

for b in bites.sort(): print(b)
# Ordem esperada na saída:
# 1 - Some n valores (Beginner)
# 37 - Reescreva um loop com recursão (Intermediate)
# 154 - Escreva uma dataclass (Intermediate)
```

In [1]:
#### Escreva sua resposta aqui
from dataclasses import dataclass, field
from enum import Enum
from typing import Any

class BiteLevel(Enum):
    Beginner = 'Beginner'
    Intermediate = 'Intermediate'
    Advanced = 'Advanced'

@dataclass(order=True)
class Bite:
    number: int
    title: str
    level: BiteLevel = field(default=BiteLevel.Beginner)
    
    def __str__(self):
        return f"{self.number} - {self.title} - ({self.level.name})"
    def __post_init__(self):
        if isinstance(self.level, str):
            self.level = BiteLevel[self.level]
bites = []
bites.append(Bite(154, 'Escreva uma dataclass', 'Intermediate'))
bites.append(Bite(1, 'Some n valores'))
bites.append(Bite(37, 'Reescreva um loop com recursão', 'Intermediate'))

bites.sort()
for b in bites: print(b)

1 - Some n valores - (Beginner)
37 - Reescreva um loop com recursão - (Intermediate)
154 - Escreva uma dataclass - (Intermediate)


#### Q2. `Pydantic`
> Adaptada desse [tutorial de Pydantic](https://github.com/adonath/scipy-2023-pydantic-tutorial/tree/main) criado por [Axel Donath](https://github.com/adonath) e [Nick Langellier](https://github.com/nlangellier).

Observe a seguinte lista de observações da previsão do tempo em Murmansk, Russia.
```python
data_samples = [
    {
        "date": "2023-05-20",
        "temperature": 62.2,
        "isCelsius": False,
        "airQualityIndex": "24",
        "sunriseTime": "01:26",
        "sunsetTime": "00:00",
    },
    {
        "date": "2023-05-21",
        "temperature": "64.4",
        "isCelsius": "not true",
        "airQualityIndex": 23,
        "sunriseTime": "01:10",
        "sunsetTime": "00:16",
    },
    {
        "date": "2023-05-22",
        "temperature": 14.4,
        "airQualityIndex": 21,
    },
]
```

Escreva um script que calcule e imprima a temperatura média (em Celsius) em Murmansk para as datas fornecidas. Em seu script, você deve incluir um modelo Pydantic que registre com sucesso todos os elementos dados. Note que:

* Algumas amostras estão faltando dados. Você deve decidir quando o atributo pode ter um valor padrão ou quando definí-lo como opcional (`typing.Optional`). 
* Você precisará implementar pelo menos um validador de campo para transformar atributos. Dica: teste primeiro quais vão falhar :)



In [8]:
#### Escreva sua resposta aqui
from pydantic import BaseModel,validator ,field_validator
from typing import Optional
from datetime import datetime,time,date

data_samples = [
    {
        "date": "2023-05-20",
        "temperature": 62.2,
        "isCelsius": False,
        "airQualityIndex": "24",
        "sunriseTime": "01:26",
        "sunsetTime": "00:00",
    },
    {
        "date": "2023-05-21",
        "temperature": "64.4",
        "isCelsius": "not true",
        "airQualityIndex": 23,
        "sunriseTime": "01:10",
        "sunsetTime": "00:16",
    },
    {
        "date": "2023-05-22",
        "temperature": 14.4,
        "airQualityIndex": 21,
    },
]

class WeatherSample(BaseModel):
    date: date
    temperature: float
    isCelsius: Optional[bool] = True
    airQualityIndex: int
    sunriseTime: Optional[time] = None
    sunsetTime: Optional[time] = None

    @field_validator('isCelsius', mode='before')
    def convert_str_to_bool(cls, v):
        if v is None:
            return True
        if isinstance(v, bool):
            return v
        if isinstance(v, str):
            v_lower = v.lower().strip()
            if v_lower in ('true', '1', 'yes', 'y'):
                return True
            elif v_lower in ('false', '0', 'no', 'not true'):
                return False
        raise ValueError(f'isCelsius inválido: {v}')

    def __post_init__(self):
        
        
        if not(self.isCelsius):
            self.temperature = (self.temperature - 32) * 5/9
    
        
def converter(i, temp, celsius,soma):
    if not celsius:
        temp = (temp - 32) * 5/9
    soma = soma + temp
    media = soma / i + 1 
    return media,soma



samples = []
for entry in data_samples:
    sample = WeatherSample(**entry)
    samples.append(sample)

soma = 0
for i, sample in enumerate(samples, 1):
    sample.temperature
    media,soma = converter(i, sample.temperature, sample.isCelsius,soma)

print(f"Temperatura média em Celsius: {media:.2f}")

Temperatura média em Celsius: 17.39


#### Q3
> Adaptada desse [tutorial de Pydantic](https://github.com/adonath/scipy-2023-pydantic-tutorial/tree/main) criado por [Axel Donath](https://github.com/adonath) e [Nick Langellier](https://github.com/nlangellier).

Na célula a seguir, coletamos dados reais de uma das principais APIs de previsão do tempo, [open-meteo](https://open-meteo.com/en/docs). Não se preocupe em entender esse código, o mais importante é entender o resultado que ele retorna, ilustrado a seguir para uma coleta da temperatura dos últimos 15 dias em Itabira -MG. Caso deseje alterar a cidade de coleta, basta alimentar a latitude e longitude desejada, como nas opções a seguir.
* Itabira: `'latitude': -19.656655787605846, 'longitude': -43.228922960534476`
* Bom Despacho: `'latitude': -19.726308457732443, 'longitude': -45.27462803349767`

```python
{
  "latitude": -19.5,
  "longitude": -43.375,
  "generationtime_ms": 0.01800060272216797,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 2.0,
  "hourly_units": {
    "time": "iso8601",
    "temperature_2m": "\u00b0C"
  },
  "hourly": {
    "time": [
      "2024-07-19T00:00",
      "2024-07-19T01:00",
      "2024-07-19T02:00",
      ...
    ],
    "temperature_2m": [
      21.9,
      20.9,
      20.0,
      ... 
    ]
  }
}
```

Você deve escrever um modelo Pydantic `OpenMeteo` que receba diretamente a resposta dessa API, através do comando:
```python
dados = OpenMeteo(**response)
``` 

Para comportar a estrutura hierárquica desse dicionário (é um dicionário com alguns dicionários internos), você deve criar uma classe Pydantic para cada dicionário interno (`HourlyUnits` e `Hourly`), com seus respectivos atributos. Essas classes serão atributos da classe principal `OpenMeteo`, que terá também os outros atributos da resposta (`latitude`, `longitude`, etc.).



In [ ]:
import requests, json
from pydantic import BaseModel
from typing import List
from datetime import datetime

url = 'https://api.open-meteo.com/v1/forecast'
lat, long = -19.656655787605846, -43.228922960534476
params = {'latitude': lat, 'longitude': long, 'elevation': 2,
          'hourly': 'temperature_2m', 'forecast_days': 15}
response = requests.get(url, params=params).json()




In [38]:
#### Escreva aqui seus modelos Pydantic

class HourlyUnits(BaseModel):
    time: str
    temperature_2m: str

class Hourly(BaseModel):
    time: List[str]
    temperature_2m: List[float]

class OpenMeteo(BaseModel):
    latitude: float
    longitude: float
    generationtime_ms: float
    utc_offset_seconds: int
    timezone: str
    timezone_abbreviation: str
    elevation: float
    hourly_units: HourlyUnits
    hourly: Hourly

dados = OpenMeteo(**response)

for d in dados: print(d)

('latitude', -19.5)
('longitude', -43.375)
('generationtime_ms', 0.02562999725341797)
('utc_offset_seconds', 0)
('timezone', 'GMT')
('timezone_abbreviation', 'GMT')
('elevation', 2.0)
('hourly_units', HourlyUnits(time='iso8601', temperature_2m='°C'))
('hourly', Hourly(time=['2025-05-14T00:00', '2025-05-14T01:00', '2025-05-14T02:00', '2025-05-14T03:00', '2025-05-14T04:00', '2025-05-14T05:00', '2025-05-14T06:00', '2025-05-14T07:00', '2025-05-14T08:00', '2025-05-14T09:00', '2025-05-14T10:00', '2025-05-14T11:00', '2025-05-14T12:00', '2025-05-14T13:00', '2025-05-14T14:00', '2025-05-14T15:00', '2025-05-14T16:00', '2025-05-14T17:00', '2025-05-14T18:00', '2025-05-14T19:00', '2025-05-14T20:00', '2025-05-14T21:00', '2025-05-14T22:00', '2025-05-14T23:00', '2025-05-15T00:00', '2025-05-15T01:00', '2025-05-15T02:00', '2025-05-15T03:00', '2025-05-15T04:00', '2025-05-15T05:00', '2025-05-15T06:00', '2025-05-15T07:00', '2025-05-15T08:00', '2025-05-15T09:00', '2025-05-15T10:00', '2025-05-15T11:00', '2025

#### Q4. 

Com os dados carregados na questão anterior plote um gráfico de linha, com a biblioteca de sua preferência, onde o eixo `x` são os timestamps (data e hora) e o eixo `y` é a temperatura medida.

In [54]:
#### Escreva aqui a sua resposta
import plotly.express as px
import pandas as pd

df = pd.DataFrame({
    'time': dados.hourly.time,
    'temperature_2m': dados.hourly.temperature_2m
})

df['time'] = pd.to_datetime(df['time'])

display(df)

fig = px.line(df, x='time', y='temperature_2m', title='Temperatura ao longo do tempo', labels={'time': 'Hora', 'temperature_2m': 'Temperatura (ºC)'} )
fig.show()

,time,temperature_2m
0,2025-05-14 00:00:00,21.2
1,2025-05-14 01:00:00,20.7
2,2025-05-14 02:00:00,20.3
3,2025-05-14 03:00:00,20.2
4,2025-05-14 04:00:00,20.4
...,...,...
355,2025-05-28 19:00:00,28.8
356,2025-05-28 20:00:00,26.1
357,2025-05-28 21:00:00,23.9
358,2025-05-28 22:00:00,22.6
